### Explain in depth your strategy to maintain and support the AIML image classifier after it in production

- Strategy Summary:
    - Please refer below cells for detailed observations
    - Considering our observed points we have to cover below details for a better maintanance approach and put it in pipeline process or with manual interventions:
        - Classification and Labeling module
            - With manual or automated approach we have to map the input images with ground truth
            - This will help us in understanding the model performance as well as creating future training data
            
        - Monitoring Module:
            - In this module we will be trackinng the model performance when we will have data available from above module.
            - Automate the process to identify the drift in model performance for weekly or monthly basis
            
        - Retraining Module:
            - Based on monitoring results we have to identify what part of model need to be trained
            - it could be just the final classifer layer for ex. if we received new species
            - It could be few additonal layers along with classifer
            - Or if we came to conclusion the whole model is turining obsolete and need a whole training
            - Identify how soon a model need to be retained
                - We should have accumulated sufficient mix of old and new data for better model performance
            
        - Deploymnet:
            - We dont want to suddenly change the model, run it in parallel with existing model for an acceptable time period and identify the performance difference
            - If the model performance is significantly improved then its predecessor, we will deploy it.

### Need for maintainance

Main Reasons for model maintainance 
- Models prediction capability fades with time if not upgraded/updated on time ( real world example - Kodak/Nokia):
    - How deep learning outperformed traditional methods in imagenet calssification, with time it surpassed even human level
    - need to devote to monitoring, validating, and tweaking it all to keep it uptodate
- Appearance of new classes (probabilistic variables):
     - CNN model can work or predict the things that it was trained to predict or to put in different words the model is only aware of the classes it was taught. If any new class came in picture it will falsly classify it to any one of the existing class.
- Data dependencies:
     - a model can work classify based on its data its being trained on.
     - with time we will be receiving/discovering new data/classes/edge case scenarios/rare species which need to be added to the train data and update the classifer with proper class
- Pipeline processes:
    - We would have designed the model and its pipeline in a way suited best during the time of creation but with time we might have to update or change that with change in new technologies and approaches.
    - In our data we are using jpg/png as immage source now but if we start getting data in some other image formats like raw image format from cameras. 
- Env/Configuration settings:
     - Upgrade in softwares - like Python versions/OS/DB
     - Upgrade in hardware - advanced new CPU/GPU/TPU
- Change of Concept:
    - with time and technolgy we may come across few ground breaking shifts in concept which might be the base of current model

##### We will try to derive our strategy keeping our Flower classification models

- When we deploy models to production and expect to observe error rates like those we saw during model evaluation, we are making an assumption that future data will be similar to past observed data. Specifically, we are assuming that the distributions of the features and targets will remain fairly constant. But this assumption usually does not hold.

- Drift In Concept:
    - Pattern of flower changes with change in climatic conditions
    - These kind of things are very slow and take long time to come in affect, we have to expand our time horizon to a very wide range to see the affect. Ex. extinction of few flowers, few flowers are getting mutated due to climate chagnes. If we are widning our horizon to cover these extents we simply dont have enough data going back but for future we might keep track of changes.
    - These slight drifts in concepts need to be identified and added in training phases to get better predictions.

- Regular Performance Metrics Evaluation & tweakings:
    - Like maintanance of any ship or software, we have to dedicate resource_hours for monitoring the performance of the on daily/weakly basis and review it. Of cource this can be traced only if we have knowledge of groundtruth which we might recieve may be after some time with manual intervention.
    - Acceptance criteria could be many - cross entropy loss, accuracy. Lets assume we took accuracy as our matrics of measure. We can keep our acceptance range baseed on project/buisness owners decisions and models capacity like 75%-95%.
    - Create acceptance criteria boundries and percentage of acceptable failures it should not cross. From above range if the average accuracy over a week is dropping towards lower boundries we need to re look into the input data its failing for and why its happening.
    - Once the model is close to testing the boundries of failures we might be entering into a phase where the model started to show its incapabilites and need tweaking or re-training or updation or addition of new classes.
    - In parallel to production, maintainance team has to carry out effort on re-training and finding new weights for the model which then can be updated for using the existing model. For this reason we can use keras model.save and model.save_weights to keep the two parts independent of each other and maintain them separately.
    - If by updating the weights is not keeping us in acceptable ranges then we have re look into the model to identify what needs to be updated. It could start from our preprocessing of images, number of layers, normalization or dropout layers or the final classification Layer.

- Monitoring a model:
    - For Monitoring we have to introduce a step where either we randomly pick samples and manually validate it with ground truths or wait for some time till the ground truths are avialabel with data.
    - We might have to introduce a step for cleaning and labelling the data to validate the prediction and simultaneously creating our data for future training purposes.
    - When we look deep into model monitoring its not just the accuracy we have to look for, we have to look for which data it failed, what was predicted and what was ground truth.
    - we have to make a proper logging mechanism which will track every prediction it has made with each input data and capture its prediction and ground truth as well. 
    - This seems easier in words but in reality we might not have actual label for a long time until it was validated or tagged or identifed manually and put into the system.

- Retraining The model:
    - Retraining a machine learning model that is already deployed in a live production environment is much easier said than done. There are multiple ways to solve a particular data-driven problem, and as we see more data our choice of the model may change.
    - If a model’s predictive performance has fallen due to changes in the environment, the solution is to retrain the model on a new training set which reflects the current reality.
    - We can consider retraining of model only after we have gathered sufficient mix of new and old data.
    - We have to also keep in mind if we want to retrain the whole model or just part of it as training whole model might be time consuming and may not be a good idea all the time.
    - Another point to consider is if we can deploy directly the new model with better accuracy or run it in parallel with old model to compare results with ground truth when available.
    - 

- Automating the Retraining process
    - If we assume the model selection is sustainable and just retraining the model and updating the weigts or redploying the whole model is viable we can use multiple approaches to automate the process.
    - We can easily use Jenkins to automate the process of training and if the model is giving significantly better result then its predecessor on identifed new Validation data, it can auto trigger the deployment process as well during off hours.
    - Triggering of training the model can be automated/manually based on accuracy metrices gathered.

- Reference
    - https://towardsdatascience.com/how-to-apply-continual-learning-to-your-machine-learning-models-4754adcd7f7f
    - https://mlinproduction.com/model-retraining/#:~:text=Rather%20retraining%20simply%20refers%20to,t%20involve%20any%20code%20changes.
    - https://machinelearningmastery.com/deploy-machine-learning-model-to-production/